In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import torch
import pickle
import random
import math
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util

import load_data
from load_data import GenderDataset, gender_data_collate_fn
from models.encoder_t5 import EncoderT5
from models.classifier_bert import ClassifierBERT
from models.perplexity_gpt import PerplexityGPT2
from models.similarity_sent_enc import encode_for_similarities

In [3]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:3')

print(device)

cuda:3


In [4]:
# Hyper parameters
num_epoch = 100
batch_size = 32
lr = 1e-4
wd = 5e-6
print_every = 200

In [5]:
with open(os.path.join(os.curdir, "data", "blog.json"), "r") as file:
    json_data = json.load(file)
docs = json_data['docs'][1:] # I don't want to see the first document

In [6]:
num_docs = len(docs)
num_train_docs = int(num_docs * 0.7)
num_val_docs = int(num_docs * 0.15)
num_test_docs = num_docs - num_train_docs - num_val_docs
print(num_train_docs, num_val_docs, num_test_docs)

13773 2951 2952


In [7]:
train_docs = docs[:num_train_docs]
val_docs = docs[num_train_docs:num_train_docs+num_val_docs]
test_docs = docs[num_train_docs+num_val_docs:]

In [8]:
train_dataset, val_dataset, test_dataset = None, None, None
load_from_pickled = False

if os.path.exists(os.path.join(os.curdir, "data", "train.pickle")):
    load_from_pickled = True
    with open(os.path.join(os.curdir, "data", "train.pickle"), "rb") as f:
        train_dataset = pickle.load(f)
    with open(os.path.join(os.curdir, "data", "val.pickle"), "rb") as f:
        val_dataset = pickle.load(f)
    with open(os.path.join(os.curdir, "data", "test.pickle"), "rb") as f:
        test_dataset = pickle.load(f)
else:
    train_dataset = GenderDataset(train_docs)
    val_dataset = GenderDataset(val_docs)
    test_dataset = GenderDataset(test_docs)

if not load_from_pickled:
    with open(os.path.join(os.curdir, "data", "train.pickle"), "wb") as f:
        pickle.dump(train_dataset, f)
    with open(os.path.join(os.curdir, "data", "val.pickle"), "wb") as f:
        pickle.dump(val_dataset, f)
    with open(os.path.join(os.curdir, "data", "test.pickle"), "wb") as f:
        pickle.dump(test_dataset, f)

print(load_from_pickled)   

True


In [9]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    collate_fn=gender_data_collate_fn
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    collate_fn=gender_data_collate_fn
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    collate_fn=gender_data_collate_fn
)

In [10]:
next(enumerate(test_dataloader))

(0,
 (tensor([[ 2982,  2001,  1041,  ..., 27162,  1015, 25273],
          [ 2295,  2012,  2007,  ...,  3475,  2045,  2049],
          [ 2011,  1016,  1015,  ...,  1049,  2576,  2009],
          ...,
          [ 1009,  1059,  2810,  ...,  1016,  1016,  1016],
          [ 5865,  1016,  6164,  ...,  1016,  2148,  2073],
          [ 2432,  4388,  2133,  ...,  2189,  1016,  2065]], dtype=torch.int32),
  tensor([128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
          128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
          128, 128, 128, 128], dtype=torch.int32),
  tensor([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
          0, 1, 0, 0, 1, 1, 1, 1])))

In [11]:
encoder_model = EncoderT5(
    vocab_size=load_data.tokenizer.vocab_size
).to(device)

classifier_model = ClassifierBERT(
    vocab_size=load_data.tokenizer.vocab_size
).to(device)

ppl_model = PerplexityGPT2(
    vocab_size=load_data.tokenizer.vocab_size,
    bos_token_id=load_data.tokenizer.bos_token_id,
    eos_token_id=load_data.tokenizer.eos_token_id,
).to(device)

st_model = SentenceTransformer('all-mpnet-base-v2')
st_model[0].auto_model=st_model[0].auto_model.to(device)

In [12]:
classifier_model.load_state_dict(torch.load('./save/cls_model_271_128_epoch_0.file'))
ppl_model.load_state_dict(torch.load('./save/ppl_model_22_48_epoch_0.file'))
encoder_model.load_state_dict(torch.load('./save/enc_model_446_32_epoch_0.file'))

<All keys matched successfully>

In [13]:
optimizer = optim.AdamW(encoder_model.parameters(), lr=lr, weight_decay=wd)

In [14]:
criterion_similarity = nn.MSELoss()
criterion_classification = nn.CrossEntropyLoss()
criterion_perplexity = nn.CrossEntropyLoss() # Placeholder

In [15]:
def train(train_dataloader, val_dataloader, models, criterions, optimizer, num_epoch):
    
    train_id = random.randint(0, 1000)
    log = open(f'./save/enc_{train_id}.txt', 'w')
    models['st'].eval()
    models['ppl'].eval()
    models['cls'].eval()

    for epoch in range(num_epoch):
        
        print(f"Epoch {epoch}, total {len(train_dataloader)} batches\n")
        log.write(f"Epoch {epoch}, total {len(train_dataloader)} batches\n")
        log.flush()

        models['enc'].train()

        for batch, (src_ids, src_len, tgt) in enumerate(train_dataloader):

            torch.cuda.empty_cache()
            optimizer.zero_grad()
            src_ids = src_ids.to(device)
            tgt = torch.full((batch_size, 2), 0.5).to(device)

            # Calculate the obfuscated logits via the encoder & softmax
            obf_logits = models['enc'](src_ids)
            obf_logits = nn.functional.softmax(obf_logits, dim=-1)
            
            # Calculate similarity loss
            result = encode_for_similarities(models['st'], device, src_ids, obf_logits)
            loss_sim = torch.mean(result)

            # Calculate gender classification loss aiming for 50/50
            gender_logits = models['cls'](obf_logits)
            loss_cls = criterions['cls'](gender_logits, tgt)
            
            # Calculate perplexity loss
            ppl_logits = models['ppl'](obf_logits)
            shift_ppl = ppl_logits[..., :-1, :].contiguous()
            shift_obf = obf_logits[..., 1:, :].contiguous()
            loss_ppl = criterions['ppl'](shift_ppl.view(-1, shift_ppl.size(-1)), shift_obf.view(-1, shift_obf.size(-1)))

            loss = loss_sim * 1 + loss_cls * 1 + loss_ppl * 1
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()
            
            if batch % print_every == 0:
                print(f"Epoch Step: {batch} Sim Loss: {loss_sim} Class Loss: {loss_cls} PPL Loss: {loss_ppl}")
                log.write(f"Epoch Step: {batch} Sim Loss: {loss_sim} Class Loss: {loss_cls} PPL Loss: {loss_ppl}\n")

                torch.save(models['enc'].state_dict(), f'./save/enc_model_{train_id}_{batch_size}_epoch_{epoch}.file')

            if batch % (print_every * 5) == 0:
                print(f"\nBegin Evaluation")
                models['enc'].eval()
                total_acc = 0
                total_ppl = 0
                total_sim = 0
                limit=len(val_dataloader)
                if limit>100: 
                    limit=100
                with torch.no_grad():
                    for batch, (src_ids, src_len, tgt) in tqdm(enumerate(val_dataloader),total=limit):
                        src_ids = src_ids.to(device)
                        tgt = tgt.to(device)

                        obf_logits = models['enc'](src_ids)
                        obf_logits = nn.functional.softmax(obf_logits, dim=-1)
                        
                        result = encode_for_similarities(models['st'], device, src_ids, obf_logits)
                        gender_logits = models['cls'](obf_logits)

                        ppl_logits = models['ppl'](obf_logits)
                        shift_ppl = ppl_logits[..., :-1, :].contiguous()
                        shift_obf = obf_logits[..., 1:, :].contiguous()
                        loss_ppl = criterions['ppl'](shift_ppl.view(-1, shift_ppl.size(-1)), shift_obf.view(-1, shift_obf.size(-1)))

                        total_acc += (gender_logits.argmax(1) == tgt).sum().item()
                        total_ppl += loss_ppl
                        total_sim += torch.mean(result)

                        if batch >= limit:
                            break

                acc = total_acc / limit / batch_size
                ppl = math.exp(total_ppl / limit)
                sim = total_sim / limit

                print(f"Validation Accuracy: {acc}\nValidation Perplexity: {ppl}\nValidation Semantic Similarity: {sim}\n")
                log.write(f"Validation Accuracy: {acc}\nValidation Perplexity: {ppl}\nValidation Semantic Similarity: {sim}\n")
                log.flush()


In [16]:
train(
    train_dataloader,
    val_dataloader,
    {
        'enc': encoder_model,
        'cls': classifier_model,
        'st': st_model,
        'ppl': ppl_model,
    },
    {
        'sim': criterion_similarity,
        'cls': criterion_classification,
        'ppl': criterion_perplexity,
    },
    optimizer,
    num_epoch
)

Epoch 0, total 14042 batches

Epoch Step: 0 Sim Loss: 0.12704554200172424 Class Loss: 1.0748761892318726 PPL Loss: 13.925819396972656

Begin Evaluation


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.66it/s]


Validation Accuracy: 0.464375
Validation Perplexity: 1247680.230880522
Validation Semantic Similarity: 0.14945170283317566

Epoch Step: 200 Sim Loss: 0.1560504138469696 Class Loss: 1.0966813564300537 PPL Loss: 1.097570538520813
Epoch Step: 400 Sim Loss: 0.1293465942144394 Class Loss: 1.0966875553131104 PPL Loss: 1.0800613164901733
Epoch Step: 600 Sim Loss: 0.12916304171085358 Class Loss: 1.0966894626617432 PPL Loss: 1.0754337310791016
Epoch Step: 800 Sim Loss: 0.17027804255485535 Class Loss: 1.0966899394989014 PPL Loss: 1.0734853744506836
Epoch Step: 1000 Sim Loss: 0.1322239190340042 Class Loss: 1.0966904163360596 PPL Loss: 1.0724605321884155

Begin Evaluation


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.60it/s]


Validation Accuracy: 0.4453125
Validation Perplexity: 2.9806030750509156
Validation Semantic Similarity: 0.1513628214597702

Epoch Step: 1200 Sim Loss: 0.15337933599948883 Class Loss: 1.0966906547546387 PPL Loss: 1.0718597173690796
Epoch Step: 1400 Sim Loss: 0.1428951472043991 Class Loss: 1.0966906547546387 PPL Loss: 1.0714712142944336
Epoch Step: 1600 Sim Loss: 0.15310955047607422 Class Loss: 1.0966906547546387 PPL Loss: 1.0711771249771118
Epoch Step: 1800 Sim Loss: 0.14789095520973206 Class Loss: 1.0966908931732178 PPL Loss: 1.0709811449050903
Epoch Step: 2000 Sim Loss: 0.1506918966770172 Class Loss: 1.0966908931732178 PPL Loss: 1.0708472728729248

Begin Evaluation


...

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.62it/s]


Validation Accuracy: 0.4534375
Validation Perplexity: 2.9478765154113726
Validation Semantic Similarity: 0.15027976036071777

Epoch Step: 4200 Sim Loss: 0.14065518975257874 Class Loss: 1.0966908931732178 PPL Loss: 1.07036554813385
Epoch Step: 4400 Sim Loss: 0.1524907648563385 Class Loss: 1.0966908931732178 PPL Loss: 1.070352554321289
Epoch Step: 4600 Sim Loss: 0.13481444120407104 Class Loss: 1.0966908931732178 PPL Loss: 1.0703411102294922
Epoch Step: 4800 Sim Loss: 0.13990692794322968 Class Loss: 1.0966908931732178 PPL Loss: 1.0703309774398804
Epoch Step: 5000 Sim Loss: 0.12830311059951782 Class Loss: 1.0966908931732178 PPL Loss: 1.0703216791152954

Begin Evaluation


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.62it/s]


Validation Accuracy: 0.4578125
Validation Perplexity: 2.9476958940128344
Validation Semantic Similarity: 0.14980584383010864

Epoch Step: 5200 Sim Loss: 0.15122868120670319 Class Loss: 1.0966908931732178 PPL Loss: 1.0703141689300537
Epoch Step: 5400 Sim Loss: 0.15208274126052856 Class Loss: 1.0966908931732178 PPL Loss: 1.0703072547912598
Epoch Step: 5600 Sim Loss: 0.15398938953876495 Class Loss: 1.0966908931732178 PPL Loss: 1.0703009366989136
Epoch Step: 5800 Sim Loss: 0.1609862744808197 Class Loss: 1.0966908931732178 PPL Loss: 1.0703173875808716


KeyboardInterrupt: 

In [15]:
def test(test_dataloader, models, criterions):
    
    models['st'].eval()
    models['ppl'].eval()
    models['cls'].eval()
    models['enc'].eval()
    
    print(f"\nBegin Testing")
    total_acc = 0
    total_ppl = 0
    total_sim = 0
    limit=len(test_dataloader)
    if limit>100: 
        limit=100
    with torch.no_grad():
        for batch, (src_ids, src_len, tgt) in tqdm(enumerate(test_dataloader),total=limit):
            src_ids = src_ids.to(device)
            tgt = tgt.to(device)

            obf_logits = models['enc'](src_ids)
            obf_logits = nn.functional.softmax(obf_logits, dim=-1)

            result = encode_for_similarities(models['st'], device, src_ids, obf_logits)
            gender_logits = models['cls'](obf_logits)

            ppl_logits = models['ppl'](obf_logits)
            shift_ppl = ppl_logits[..., :-1, :].contiguous()
            shift_obf = obf_logits[..., 1:, :].contiguous()
            loss_ppl = criterions['ppl'](shift_ppl.view(-1, shift_ppl.size(-1)), shift_obf.view(-1, shift_obf.size(-1)))

            print(test_dataset.idx2str(src_ids[0].cpu().detach()))
            print(test_dataset.idx2str(obf_logits[0].argmax(-1).cpu().detach()))

            total_acc += (gender_logits.argmax(1) == tgt).sum().item()
            total_ppl += loss_ppl
            total_sim += torch.mean(result)

            if batch >= limit:
                break

    acc = total_acc / limit / batch_size
    ppl = math.exp(total_ppl / limit)
    sim = total_sim / limit

    print(f"Test Accuracy: {acc}\nTest Perplexity: {ppl}\nTest Semantic Similarity: {sim}\n")

In [16]:
test(
    test_dataloader,
    {
        'enc': encoder_model,
        'cls': classifier_model,
        'st': st_model,
        'ppl': ppl_model,
    },
    {
        'sim': criterion_similarity,
        'cls': criterion_classification,
        'ppl': criterion_perplexity,
    }
)


Begin Testing


  1%|▊                                                                                  | 1/100 [00:01<02:12,  1.34s/it]

for all the readers of this post be notifiyed that anything said has nothing to do with me even if it does 5 days in bali last week i got 2 rings one for a sovinior one for giving away to someone someone speacial whos a girl who would have done her o - levels who would not have a bf who would not be around me for crap who... ect... ect... ect for those friends reading - lrb - espeacially the girls - rrb - please take note the top ^ ^ nothing to do with you guys...
##88888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888


  2%|█▋                                                                                 | 2/100 [00:01<01:09,  1.41it/s]

i generally don't promote products, but i find i have to make the exception for this : neutrogena blackhead eliminating - lrb - tm - rrb - scrub now, while i have been blessed with skin that never had bad acne, the occassional break - out every six months or so, of late i have bemoaning the pores on my nose. the super - gross - clogged - with - gunk pores on my nose. they're not really blackheads, they're just icky. those biore strips that claim to clear out your
##88888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888


...

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.68it/s]

did get some really great stuff and i think her film will look pretty good. i also spent some time in carbondale on my layover to and from nashville. it was nice. i felt awkward at first being at the school i just left, but i got to see some old friends and i had a great time with them. i also got to go eat at quatros. which is the best pizza ever. i even had chocolate chip cheesecake. yum. one of my friends got a dui this weekend. i am usually really unsympathetic when people drink and drive and get caught, but i love
##88888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.53it/s]

with my mom for me and he called the day i got in and told mehespent had spent valentines day at his sisters playing with his nephew because hecouldn't have imagined being with anyonebut me sohe stayed in. the last time we talked was earlyjune he was getting ready for graduation. soi was only 15 andididn't and still don'tknow anything at all about love and romance butit was that perfect week of my life. if we never talk again i'll be satisfied with how everythingwentand i'll be happy that it happened.
##88888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888


Test Accuracy: 0.5353125
Test Perplexity: 2.277333948288097
Test Semantic Similarity: 0.1469321846961975

